In [1]:
# Import necessary libraries
import jax.numpy as jnp
from jax import random, grad, jit, lax, jacrev
from jax.scipy.linalg import inv, svd, eigh, det
from jax.numpy.linalg import norm
from tqdm.auto import tqdm
from sklearn.datasets import make_spd_matrix
from jax_models import visualize_observations, Lorenz96, KuramotoSivashinsky, generate_true_states, generate_localization_matrix
from jax_filters import ensrf_steps, kalman_filter_process, ensrf_step
import jax
import matplotlib.pyplot as plt
from jax.tree_util import Partial
from functools import partial
from jax_vi import KL_gaussian, log_likelihood, KL_sum
jax.config.update("jax_enable_x64", True)
#jax.config.update("jax_debug_nans", True)

N = 10 # number of Monte Carlo samples
F = 8.0
dt = 0.05
num_steps = 250  # Number of time steps
J0 = 0
n = 40   # Number of state variables
key = random.PRNGKey(0)  # Random key for reproducibility
x0 = random.normal(key, (n,))
initial_state  = x0
# Noise covariances
Q = 0.03 * jnp.eye(n)  # Process noise covariance
R = 0.5 * jnp.eye(n)  # Observation noise covariance
# Observation matrix (identity matrix for direct observation of all state variables)
H = jnp.eye(n)

n_ensemble = 5
observation_interval = 1
initial_state = random.normal(random.PRNGKey(0), (n,))  # Initial state
m0 = initial_state
C0 = Q


l96_model = Lorenz96(dt = dt, F = 8)
l96_step = Partial(l96_model.step)
# Generate true states and observations using the Lorenz '96 model

state_transition_function = l96_model.step

jacobian_function = jacrev(l96_step, argnums=0)
jac_func = Partial(jacobian_function)

observations, true_states = generate_true_states(key, num_steps, n, initial_state, H, Q, R, l96_step, observation_interval)

In [2]:
@jit
def var_cost(radius, inflation, model, ensemble_init, observations, H, Q, R, key, J, J0):
    localization_matrix = generate_localization_matrix(n, radius)
    states, covariances = ensrf_steps(l96_step, n_ensemble, ensemble_init, num_steps, observations, observation_interval, H, Q, R, localization_matrix, inflation, key)
    ensemble_mean = jnp.mean(states, axis=-1)  # Taking the mean across the ensemble members dimension
    key, *subkeys = random.split(key, num=N+1)
    kl_sum = KL_sum(ensemble_mean, covariances, n, model, Q, key, N)
    def inner_map(subkey):
        return log_likelihood(random.multivariate_normal(subkey, ensemble_mean, covariances), observations, H, R, J, J0)  # Sometimes the covariances are negative definite. Fix
    cost = kl_sum - jnp.mean(jax.lax.map(inner_map, jnp.vstack(subkeys)))
    return cost

In [3]:
inflation = 1.5  # Fixed starting value for inflation
radius_opt = 1.0
alpha = 1e-4  # Learning rate
key = random.PRNGKey(0)  # Random key
N = 10  # Number of MC samples
m0 = initial_state
C0 = Q  # Initial covariance, assuming Q is your process noise covariance
ensemble_init = random.multivariate_normal(key, initial_state, Q, (n_ensemble,)).T
localization_matrix = generate_localization_matrix(n, radius_opt)

states, covariances = ensrf_steps(l96_step, n_ensemble, ensemble_init, num_steps, observations, observation_interval, H, Q, R, localization_matrix, inflation, key)


In [40]:
key, subkey = random.split(key)
var_cost(radius_opt, inflation, l96_step, ensemble_init, observations, H, Q, R, subkey, num_steps, J0)

Array(119137.01306575, dtype=float64)

In [16]:
print(jax.value_and_grad(var_cost) (radius_opt, inflation, l96_step, ensemble_init, observations, H, Q, R, subkey, num_steps, J0))

(Array(25045.32123712, dtype=float64), Array(-8381.46181025, dtype=float64))


In [42]:
ensemble_init.shape

(40, 5)

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

n_ensemble = 5
ensemble_init = random.multivariate_normal(key, initial_state, Q, (n_ensemble,)).T

radius_range = jnp.arange(1, 41, 2)  
inflation_range = jnp.linspace(1.01, 1.9, 20) 

radius_values, inflation_values, cost_values = [], [], []

# Compute the cost for each combination of radius_opt and inflation
for radius_opt in radius_range:
    for infl in inflation_range:
        key, subkey = random.split(key)
        cost = var_cost(radius_opt, infl, l96_step, ensemble_init, observations, H, Q, R, subkey, num_steps, J0)
        radius_values.append(radius_opt)
        inflation_values.append(infl)
        cost_values.append(cost)

radius_values = jnp.array(radius_values)
inflation_values = jnp.array(inflation_values)
cost_values = jnp.array(cost_values)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

scat = ax.scatter(radius_values, inflation_values, cost_values, c=cost_values, cmap='viridis')
ax.set_xlabel('Radius')
ax.set_ylabel('Inflation')
ax.set_zlabel('Cost')
ax.set_title('Cost Surface Across Radius and Inflation')

cbar = fig.colorbar(scat, ax=ax, pad=0.1)
cbar.set_label('Cost')

plt.show()
